# Azure AI Document Intelligence - Document Analysis

Dieses Notebook zeigt, wie man Azure AI Document Intelligence (Form Recognizer) verwendet, um:
- Dokumente aus Blob Storage zu laden
- Layout zu analysieren
- Text zu extrahieren
- Tabellen zu extrahieren
- Absätze zu extrahieren

## 1. Installation

In [ ]:
# Pakete installieren
!uv pip install python-dotenv azure-ai-formrecognizer azure-storage-blob ipywidgets

## 2. Umgebungsvariablen laden

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

env_path = Path('..') / '..' / '.env'
load_dotenv(env_path)

print('Konfiguration geladen:')
print(f"  • Document Intelligence Endpoint: {os.getenv('DOCUMENT_INTELLIGENCE_ENDPOINT')}")
print(f"  • Storage Container: {os.getenv('DOC_INTEL_CONTAINER_NAME')}")

## 3. Document Intelligence Client initialisieren

In [ ]:
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

endpoint = os.getenv('DOCUMENT_INTELLIGENCE_ENDPOINT')
api_key = os.getenv('DOCUMENT_INTELLIGENCE_API_KEY')

if not endpoint or not api_key:
    raise ValueError('DOCUMENT_INTELLIGENCE_ENDPOINT und DOCUMENT_INTELLIGENCE_API_KEY müssen in .env gesetzt sein')

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(api_key)
)

print('✓ Document Analysis Client initialisiert!')

## 4. Blob Storage Client initialisieren

In [ ]:
from azure.storage.blob import BlobServiceClient

storage_connection_string = os.getenv('DOC_INTEL_STORAGE_CONNECTION_STRING')
container_name = os.getenv('DOC_INTEL_CONTAINER_NAME')

if not storage_connection_string or not container_name:
    raise ValueError('DOC_INTEL_STORAGE_CONNECTION_STRING und DOC_INTEL_CONTAINER_NAME müssen in .env gesetzt sein')

blob_service_client = BlobServiceClient.from_connection_string(storage_connection_string)
container_client = blob_service_client.get_container_client(container_name)

print('✓ Blob Storage Client initialisiert!')

## 5. Dokumente im Blob Storage auflisten

In [ ]:
print('Dokumente im Blob Storage:')
print('=' * 80)

blobs = list(container_client.list_blobs())

if not blobs:
    print('⚠️  Keine Dokumente gefunden!')
    print('   Bitte lade zuerst Dokumente hoch mit:')
    print('   python3 tools_and_data/document_intelligence/examples/upload_sample_data.py')
else:
    for idx, blob in enumerate(blobs, start=1):
        size_kb = blob.size / 1024
        print(f"{idx}. {blob.name} ({size_kb:.2f} KB)")

## 6. Erstes Dokument analysieren

Wir analysieren das erste Dokument mit dem `prebuilt-layout` Modell.

In [ ]:
if blobs:
    first_blob = blobs[1]
    print(f'Analysiere Dokument: {first_blob.name}')
    print('=' * 80)
    
    # Hole Blob URL
    blob_client = container_client.get_blob_client(first_blob.name)
    blob_url = blob_client.url
    
    print(f'Blob URL: {blob_url}')
    print()
    
    # Analysiere mit Layout-Modell
    print('Starte Analyse...')
    poller = document_analysis_client.begin_analyze_document_from_url(
        model_id="prebuilt-layout",
        document_url=blob_url
    )
    
    result = poller.result()
    
    print('✓ Analyse abgeschlossen!')
    print()
    print(f'Seiten: {len(result.pages)}')
    print(f'Tabellen: {len(result.tables)}')
    print(f'Absätze: {len(result.paragraphs)}')
else:
    print('Keine Dokumente zum Analysieren vorhanden.')

## 7. Text extrahieren

Extrahiere den gesamten Text aus dem Dokument.

In [ ]:
if blobs and 'result' in locals():
    print('Extrahierter Text:')
    print('=' * 80)
    
    for page_idx, page in enumerate(result.pages, start=1):
        print(f'\n--- Seite {page_idx} ---')
        print(f'Größe: {page.width} x {page.height} {page.unit}')
        print(f'Zeilen: {len(page.lines)}')
        print()
        
        for line in page.lines:
            print(line.content)

## 8. Tabellen extrahieren

Extrahiere alle Tabellen aus dem Dokument.

In [ ]:
if blobs and 'result' in locals():
    print('Tabellen:')
    print('=' * 80)
    
    if not result.tables:
        print('Keine Tabellen gefunden.')
    else:
        for table_idx, table in enumerate(result.tables, start=1):
            print(f'\nTabelle {table_idx}:')
            print(f'  Zeilen: {table.row_count}')
            print(f'  Spalten: {table.column_count}')
            print()
            
            # Erstelle eine Matrix für die Tabelle
            matrix = [['' for _ in range(table.column_count)] for _ in range(table.row_count)]
            
            for cell in table.cells:
                matrix[cell.row_index][cell.column_index] = cell.content
            
            # Ausgabe der Tabelle
            for row in matrix:
                print('  | ' + ' | '.join(row) + ' |')

## 9. Absätze extrahieren

Extrahiere strukturierte Absätze aus dem Dokument.

In [ ]:
if blobs and 'result' in locals():
    print('Absätze:')
    print('=' * 80)
    
    if not result.paragraphs:
        print('Keine Absätze gefunden.')
    else:
        for para_idx, paragraph in enumerate(result.paragraphs, start=1):
            print(f'\nAbsatz {para_idx}:')
            print(f'  Rolle: {paragraph.role if paragraph.role else "N/A"}')
            content = paragraph.content
            if len(content) > 100:
                print(f'  Inhalt: {content[:100]}...')
            else:
                print(f'  Inhalt: {content}')

## 10. Zusammenfassung

In [ ]:
if blobs and 'result' in locals():
    print('Analyse-Zusammenfassung:')
    print('=' * 80)
    print(f'Dokument: {first_blob.name}')
    print(f'Seiten: {len(result.pages)}')
    print(f'Tabellen: {len(result.tables)}')
    print(f'Absätze: {len(result.paragraphs)}')
    print()
    
    # Zähle Textzeilen
    total_lines = sum(len(page.lines) for page in result.pages)
    print(f'Textzeilen: {total_lines}')
    
    # Zähle Tabellenzellen
    total_cells = sum(len(table.cells) for table in result.tables)
    print(f'Tabellenzellen: {total_cells}')
    print()
    print('✓ Analyse erfolgreich abgeschlossen!')

## Nächste Schritte

- Probiere weitere Dokumente aus dem Blob Storage
- Teste das zweite Notebook für Prebuilt Models: `02_prebuilt_models.ipynb`
- Lade eigene Dokumente hoch und analysiere sie